In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('drive/My Drive/signate/takeda/dataset/train.csv')
dftest = pd.read_csv('drive/My Drive/signate/takeda/dataset/test.csv')

print(df.shape)
print(dftest.shape)

(13731, 3807)
(13732, 3806)


In [0]:
df_Score = df['Score']
df = df.drop('Score', axis=1)
dftest_id = dftest['ID']

print(df.shape)
print(dftest.shape)

(13731, 3806)
(13732, 3806)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

X = df.as_matrix()
y = df_Score.as_matrix()

clf = lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_depth=17,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=15000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

clf.fit(X, y)

XX = dftest.as_matrix()

dftest_Score = pd.DataFrame(clf.predict(XX))

dftest_Score = pd.concat(objs=[dftest_id, dftest_Score], axis=1)
dftest_Score.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


,ID,0
0,1,1.215885
1,2,2.882384
2,3,2.552055
3,4,1.638556
4,6,3.098467


In [0]:
dftest_Score.to_csv('submit_LGB.csv', index=False, header=False)

In [0]:
# 終了時に音
import numpy as np
import IPython as IP
fs = 48000
duration = 3.0
def play_sine(f):
    t = np.linspace(0., duration, int(fs * duration))  # ( start, stop, num of data )
    x = np.sin(f * (2. * np.pi) * t)
    IP.display.display(IP.display.Audio(x, rate=fs, autoplay=True))
play_sine(440)

In [0]:
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV

#import lightgbm as lgb
#from sklearn.ensemble import GradientBoostingRegressor
#from sklearn import linear_model
#from sklearn.linear_model import LinearRegression
#import xgboost as xgb
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import Ridge
#from sklearn.ensemble import RandomForestRegressor

#X = df.as_matrix()
#y = df_Score.as_matrix()

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

#clf = lgb.LGBMRegressor()

#parameters = {'max_depth': [17, 18, 19, 20],
#              'learning_rate': [0.01],
#              'n_estimators': [7500, 10000, 12500]}

#model = GridSearchCV(clf,
#                     parameters,
#                     cv=5)

#clf = lgb.LGBMRegressor()         # 0.540 # 0.537
#clf = GradientBoostingRegressor() # 0.472
#clf = xgb.XGBRegressor()          # 0.472
#clf = RandomForestRegressor()     # 0.441 

#model.fit(X_train, y_train)

#print('決定係数(train):{:.3f}'.format(clf.score(X_train, y_train)))
#print('決定係数(test):{:.3f}'.format(clf.score(X_test, y_test)))
#print('決定係数(train):{:.3f}'.format(model.score(X_train, y_train)))
#print('決定係数(test):{:.3f}'.format(model.score(X_test, y_test)))

In [0]:
#print(model.best_estimator_)

In [0]:
# 欠損値の処理
df_missing_value = pd.DataFrame()
dftest_missing_value = pd.DataFrame()

for flag, name in zip(df.isnull().any(), df.columns):
  if flag == True:
    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
    df = df.drop(name, axis=1)
    dftest = dftest.drop(name, axis=1)
    
for flag, name in zip(dftest.isnull().any(), dftest.columns):
  if flag == True:
    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
    dftest = dftest.drop(name, axis=1)
    df = df.drop(name, axis=1)

same_colums = []
same_columns = df_missing_value.columns & dftest_missing_value.columns
print(len(same_colums))
print(df_missing_value.shape)
print(dftest_missing_value.shape)

0
(0, 0)
(0, 0)
